# Story Generator - Winnie the Pooh 

## Part 2: Simple DSPy Retriever and Module

[1. Imports and environment](#1-imports-and-environment)

[2. Chroma retriever](#2-chroma-retriever)

[3. DSPy module](#3-dspy-module)

[4. Testing StoryGenerator](#4-testing-storygenerator)

[5. Evaluation metrics - readability](#5-evaluation-metrics---readability)

### 1. Imports and environment

In [1]:
#pip install dspy-ai openai chromadb sentence_transformers spacy textstat

In [2]:
import dspy
from dspy.retrieve.chromadb_rm import ChromadbRM
import chromadb
from chromadb.utils import embedding_functions
import dotenv
import os
import spacy


# Establish paths
CHROMA_PATH = '../data/chroma_db'
DB_COLLECTION = "winnie_the_pooh"
default_ef = embedding_functions.DefaultEmbeddingFunction()

# Set up OpenAI API key
dotenv.load_dotenv()
#openai_key = os.getenv('OPENAI_API_KEY')

True

In [3]:
# List all collections in the Chroma database
chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)
collections = chroma_client.list_collections()
print(collections)

[Collection(id=931a636a-d481-48d4-a1b8-19eecdac5b6f, name=winnie_the_pooh)]


### 2. Chroma retriever

In [4]:
# Configure OpenAI as the language model
llm = dspy.OpenAI(model="gpt-4o-mini", max_tokens=1000, temperature=1.0)

# Set up Chroma client and retriever
chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)
collection = chroma_client.get_collection(DB_COLLECTION)

# Set up ChromadbRM as the retriever model
chroma_retriever = ChromadbRM(
    collection_name=DB_COLLECTION, 
    persist_directory=CHROMA_PATH, 
    embedding_function=default_ef,
    )

# Configure DSPy settings
dspy.settings.configure(lm=llm, rm=chroma_retriever)

In [5]:
#example of calling retriever
results = chroma_retriever("honey")
len(results)
results[0]

{'id': '4ed6d226-f7d4-42eb-be32-fab74ffeea4c',
 'score': 1.08407461643219,
 'long_text': 'then he got up, and said: "And the only reason for making honey is so as I can eat it." So he began to climb the tree. He climbed and he climbed and he climbed, and as he climbed he sang a little',
 'metadatas': {'author': 'A. A. Milne',
  'chapter': 1.0,
  'chunk': 17.0,
  'title': 'Winnie the Pooh'}}

### 3. DSPy module

In [6]:

class GenerateStory(dspy.Signature):
    """Generate a Winnie the Pooh style story."""
    name = dspy.InputField()
    prompt = dspy.InputField(desc="details to include in the story.")
    context = dspy.InputField(desc="relevant passages from Winnie the Pooh stories and story structure.")
    story = dspy.OutputField(desc="generate a one-minute story for a child. Name is the main character who is friends with Pooh, and finish the story with 'The End.'")


class StoryGenerator(dspy.Module):
    def __init__(self, chroma_retriever):
        super().__init__()
        self.retriever = chroma_retriever
        self.generate = dspy.ChainOfThought(GenerateStory)

    def forward(self, name, prompt):
        retrieved = self.retriever(prompt, k=8)
        retrieved_context = [doc.long_text for doc in retrieved]
        context = "\n".join(retrieved_context)
        
        result = self.generate(context=context, prompt=prompt, name=name)
        return dspy.Prediction(story=result.story)



### 4. Testing StoryGenerator 

In [7]:
# Create an instance of the StoryGenerator
story_gen = StoryGenerator(chroma_retriever)


In [8]:
name= 'Rose'
prompt = "They go into town and meet many new friends and have a picnic."

new_story2 = story_gen(name, prompt)
print(new_story2.story)

 		You are using the client GPT3, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb


Once upon a time in the Hundred Acre Wood, Rose, a cheerful little girl with rosy cheeks, was playing with her best friend, Winnie the Pooh. One sunny morning, Rose exclaimed excitedly, "Pooh, let’s go to town and meet new friends!"

"That sounds splendid!" replied Pooh, his tummy rumbling at the thought of all the honey he might find. So off they went, skipping along the path, hand in hand.

When they arrived in town, they saw many bustling animals: rabbits hopping, birds chirping, and even a friendly little deer who waved at them. "Hello!" shouted Rose. "What a lovely day to make friends!"

"Would you like to have a picnic?" Pooh suggested with a honeyed grin. Rose agreed, and together they collected acorns, berries, and, of course, a big pot of honey from Pooh’s secret stash.

They found a nice sunny spot beneath a grand oak tree. Soon, more of their friends joined them — Piglet, Eeyore, and Tigger bounced over, each bringing their favorite snacks. 

As they spread the blanket on th